In [1]:
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.local import LocalQuantumJob
import time

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [2]:
num_total_updates = 10

hyperparams = {
    "num_total_updates": num_total_updates, 
}

#Name your job so that it can be later identified
job_name = f'cx-gate-calibration-{num_total_updates}-updates'

In [3]:
job = AwsQuantumJob.create(
    #Run this hybrid job on the SV1 simulator
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    #The directory or single file containing the code to run.
    source_module="algorithm_script.py",
    #The main script or function the job will run.
    entry_point="algorithm_script:calibrate_gate",
    #Set the name of the job
    job_name=job_name,
    # S3 Location of the files needed for the job
    code_location='s3://amazon-braket-us-west-1-lukasvoss/jobs/cx-gate_calibration-10-updates-02/needed_files/',
    # Set the hyperparameters
    hyperparameters=hyperparams,
    # Define the file that contains the input data
    input_data="config_yamls/agent_config.yaml",
    wait_until_complete=True,
    aws_session=aws_session,
)

ClientError: An error occurred (ResourceConflictException) when calling the CreateJob operation: A job with the name cx-gate-calibration-10-updates-02 already exists in this account. Please provide a different name.

In [ ]:
while job.state() not in job.TERMINAL_STATES:
    print(job.state())
    time.sleep(30)

print(job.state())
print(job.result())